In [73]:
#imports and basic port/camera settings
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")

#Sets the hdmi config to the display
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()
frame_out_w = 1920
frame_out_h = 1080

# camera (input) configuration
frame_in_w = 640
frame_in_h = 480


import os
os.environ["OPENCV_LOG_LEVEL"]="SILENT"
# initialize camera from OpenCV
import cv2
# import cv2
import numpy as np
# import matplotlib.pyplot as plt
import random as rand
import matplotlib.image as img # used to get images

# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sb
# import random as rand
# import cv2
# import matplotlib.image as img

In [74]:
webcam = cv2.VideoCapture(0)
webcam.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
webcam.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);

print("Capture device is open: " + str(webcam.isOpened()))

#-------------------------

## Parameter to change outputted password length

password_length = 20

Capture device is open: True


In [ ]:
#in order to make the camera take photos regularly
import time
# Capture webcam image
import numpy as np

%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np

rval, frame = webcam.read()

print("While Loop Test")
while rval:
    print("Taking Pictures...")
#     time.sleep(5)

    # --------------
    failed_pics = 0
    # Reading raw lava lamp captures
    rval, frame = webcam.read()
    if(rval == 0):
        failed_pics = 1
    pic_1 = frame
    rval, frame = webcam.read()
    if(rval == 0):
        failed_pics = 1
    pic_2 = frame
    rval, frame = webcam.read()
    if(rval == 0):
        failed_pics = 1
    pic_3 = frame
    rval, frame = webcam.read()
    if(rval == 0):
        failed_pics = 1
    pic_4 = frame
    
    if(failed_pics):
        print("Failed taking a picture!")
    
    ## Encoding movements of lava lamps

    def rgb2gray(rgb):
        return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

    # Converting to images gray scale
    g1 = rgb2gray(pic_1)
    g2 = rgb2gray(pic_2)
    g3 = rgb2gray(pic_3)
    g4 = rgb2gray(pic_4)
    # plt.imshow(g2, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
    print("Converted to grey scale")
    # Determining movements over time period
    diff1 = cv2.absdiff(g2, g3)
    diff2 = cv2.absdiff(g1, g4)
    # plt.imshow(diff1, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
    print("Abs Diff Done")
    # Combining movements into 3D tensor(3rd dimension is time)
    diff_max = np.dstack((diff1, diff2))
    print("Dstack done")
    
    # Approximating data matrices with low-rank ones, while still preserving variance. 
    # Used heavily in data mining to find features in higher order matrixes.
    # For our purposes, we transform the movement of lava lamps in 3D(x,y,time) into a single matrix for feature extraction.

    print("Starting SVD...")
#     print(diff_max.shape)
    u, s, vh = np.linalg.svd(diff_max[200:215])
    print("Finished SVD...")
    
    # Reservoir Sampling Algorithim
    # Mathmatically proven to sample M points in an equal distribution from a given list/stream.

    print("Reservior Sampling Starting...")
    def reservoir_sampling(stream, M):
        sampled = [-1]*M
        for i in range(len(stream)):
            if(i < M): #Space avaliable
                sampled[i] = stream[i]
            else:
                draw = rand.randint(0,i)
                if(draw < M):
                    sampled[draw] = stream[i]
        return sampled
        
    print("Reservior Sampling Finished...")
    
    allChars = "qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM1234567890!@#$%^&*()=+\<>/?"
    shuffledAllChars = "dE=Ahe\jFfN2DB+x#$5qSyVmg/pUoP@sHQW(anCG&rzw^)J?k%KliZ<YcbvtX>9T!uRL6I014M8O37*"

    print("Sampling...")
    # Sampling values from low-rank matrix, and turning them into index values.
    prestreamed = []
    for i in range(int(s.size/2)):
        if(s[i][0] > 0):
            prestreamed.append(int(s[i][0] * 1000) % len(shuffledAllChars))
        if(s[i][1] > 0):
            prestreamed.append(int(s[i][1] * 1000) % len(shuffledAllChars))

    indexes = reservoir_sampling(prestreamed, password_length)
    print("Sampling Done")
    
    # Index values will be used to select characters from 'shuffledAllChars' variable
    # and putting them into a password variable, which will then be outputted.  

    password = ""
    for i in range(len(indexes)):
        password += (shuffledAllChars[indexes[i]])
        
    print("True Random Password:", password)
    #---------------
    #outputting image on display
    display = hdmi_out.newframe()
    #display[0:480,0:640,:] = frame[0:480,0:640,:]
    
    
#     image = pic_3
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (140,145)
    fontScales = 1
    color = (255,0,0)
    thickness = 2
#     (textW, textH) = cv2.getTextSize(password,font,fontScale=fontScales,thickness=1)
    
#     startX = (frame.shape[1]-textW)//2
#     startY = (frame.shape[0]-textH)//2
    image = cv2.putText(frame,password,org,font,fontScale,color,thickness,cv2.LINE_AA, False)
#     image = cv2.putText(frame,password,org,font,fontScale,color,thickness,cv2.LINE_AA, True)
    display[0:480,0:640,:] = frame[0:480,0:640,:]
#     print(frame.shape)
#     rval, frame = webcam.read()
    hdmi_out.writeframe(display)
#     hdmi_out.writeframe(display)
    time.sleep(10)
    
    #outputting image as jpeg on jupyter
#     plt.imshow(frame[:,:,[2,1,0]])
#     plt.show()
    key = cv2.waitKey(1)
    if key % 256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
else:
     raise RuntimeError("Failed to read from camera.")
        
print("End of While Loop")

#release camera and hdmi
webcam.release()
hdmi_out.stop()
del hdmi_out

While Loop Test
Taking Pictures...
Converted to grey scale
Abs Diff Done
Dstack done
Starting SVD...
Finished SVD...
Reservior Sampling Starting...
Reservior Sampling Finished...
Sampling...
Sampling Done
True Random Password: OuaZt0v$=@oNZY1NuWHI
Taking Pictures...
Converted to grey scale
Abs Diff Done
Dstack done
Starting SVD...
Finished SVD...
Reservior Sampling Starting...
Reservior Sampling Finished...
Sampling...
Sampling Done
True Random Password: S?FOf8&hJ4A8L?aTOlXG
Taking Pictures...
Converted to grey scale
Abs Diff Done
Dstack done
Starting SVD...
Finished SVD...
Reservior Sampling Starting...
Reservior Sampling Finished...
Sampling...
Sampling Done
True Random Password: f\35wcCl6hWQS1=wm3Em
Taking Pictures...
Converted to grey scale
Abs Diff Done
Dstack done
Starting SVD...
Finished SVD...
Reservior Sampling Starting...
Reservior Sampling Finished...
Sampling...
Sampling Done
True Random Password: 24sb5y1agL?ZL3dbd7+4
Taking Pictures...
Converted to grey scale
Abs Diff Done